In [1]:
import pandas as pd
import numpy as np
import copy
import sys
import os
sys.path.append("../")
from parser.utils import load_json, dfs_cardinality, estimate_scan_in_mb
from models.feature.single_xgboost_feature import find_top_k_operators, featurize_one_plan, get_top_k_table_by_size
from utils.load_brad_trace import load_trace, create_concurrency_dataset, load_trace_all_version
from models.concurrency.utils import pre_info_train_test_seperation
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from models.single.stage import SingleStage
from models.concurrency.complex_models import ConcurrentRNN
np.set_printoptions(suppress=True)

In [2]:
parsed_queries_path = "/Users/ziniuw/Desktop/research/Data/AWS_trace/mixed_aurora/aurora_mixed_parsed_queries.json"
plans = load_json(parsed_queries_path, namespace=False)
folder_name = "mixed_aurora"
directory = f"/Users/ziniuw/Desktop/research/Data/AWS_trace/{folder_name}/"
all_raw_trace, all_trace = load_trace_all_version(directory, 8, concat=True)
all_concurrency_df = []
for trace in all_trace:
    concurrency_df = create_concurrency_dataset(trace, engine=None, pre_exec_interval=200)
    all_concurrency_df.append(concurrency_df)
concurrency_df = pd.concat(all_concurrency_df, ignore_index=True)

In [3]:
train_trace_df_sep, eval_trace_df_sep = pre_info_train_test_seperation(concurrency_df)
print(len(train_trace_df_sep), len(eval_trace_df_sep))
np.random.seed(0)
train_idx = np.random.choice(len(concurrency_df), size=int(0.8 * len(concurrency_df)), replace=False)
test_idx = [i for i in range(len(concurrency_df)) if i not in train_idx]
train_trace_df = copy.deepcopy(concurrency_df.iloc[train_idx])
eval_trace_df = concurrency_df.iloc[test_idx]
eval_trace_df = copy.deepcopy(eval_trace_df[eval_trace_df['num_concurrent_queries'] > 0])
print(len(train_trace_df), len(eval_trace_df))

28925 25561
43967 10907


In [4]:
ss = SingleStage(use_table_features=True, true_card=True)
#df = ss.featurize_data(train_trace_df, parsed_queries_path)
df = ss.featurize_data(concurrency_df, parsed_queries_path)
ss.train(df)

Top 20 operators contains 0.9650782102582758 total operators


In [35]:
rnn = ConcurrentRNN(ss, 
                    input_size=len(ss.all_feature[0]) * 2 + 7,
                    embedding_dim=128,
                    hidden_size=256,
                    num_layers=2,
                    loss_function="q_loss",
                    last_output=True,
                    use_seperation=False
                   )
rnn.train(train_trace_df_sep, eval_trace_df_sep, lr=0.001, loss_function="q_loss", val_on_test=True)
#rnn.load_model("checkpoints")

********Epoch 0, training loss: 337.70144646463143 || evaluation loss: ********


100%|████████████████████████████████████████| 200/200 [00:01<00:00, 165.05it/s]


50% absolute error is 4.593544006347656, q-error is 4.391744136810303
90% absolute error is 18.597309112548828, q-error is 36.16236877441406
95% absolute error is 32.60484313964844, q-error is 67.83975219726562
********Epoch 5, training loss: 58.69108995247995 || evaluation loss: ********


100%|████████████████████████████████████████| 200/200 [00:01<00:00, 162.93it/s]


50% absolute error is 3.923961877822876, q-error is 4.590317249298096
90% absolute error is 50.619380950927734, q-error is 57.53612518310547
95% absolute error is 75.27839660644531, q-error is 123.46634674072266
********Epoch 10, training loss: 0.3143513170359409 || evaluation loss: ********


...

100%|████████████████████████████████████████| 200/200 [00:01<00:00, 154.44it/s]


50% absolute error is 11.477429389953613, q-error is 7.534438133239746
90% absolute error is 95.69139099121094, q-error is 137.40350341796875
95% absolute error is 129.03465270996094, q-error is 316.54718017578125
********Epoch 115, training loss: 6.15091284107318 || evaluation loss: ********


100%|████████████████████████████████████████| 200/200 [00:01<00:00, 151.03it/s]


50% absolute error is 12.527532577514648, q-error is 7.842449188232422
90% absolute error is 103.46224975585938, q-error is 153.44252014160156
95% absolute error is 138.62750244140625, q-error is 357.5441589355469


KeyboardInterrupt: 

In [34]:
rnn2.save_model("checkpoints")

In [43]:
rnn2 = ConcurrentRNN(ss, 
                    input_size=len(ss.all_feature[0]) * 2 + 7,
                    embedding_dim=128,
                    hidden_size=256,
                    num_layers=2,
                    loss_function="q_loss",
                    last_output=True,
                    use_seperation=True
                   )
rnn2.train(train_trace_df_sep, eval_trace_df_sep, lr=0.001, loss_function="q_loss", val_on_test=True)

********Epoch 0, training loss: 148.37454906870835 || evaluation loss: ********


100%|████████████████████████████████████████| 200/200 [00:01<00:00, 146.33it/s]


50% absolute error is 5.457521438598633, q-error is 4.738193511962891
90% absolute error is 23.25655746459961, q-error is 45.17656326293945
95% absolute error is 36.934661865234375, q-error is 86.25215148925781
********Epoch 5, training loss: 0.3641954805471201 || evaluation loss: ********


100%|████████████████████████████████████████| 200/200 [00:01<00:00, 132.25it/s]


50% absolute error is 3.4776265621185303, q-error is 4.019598484039307
90% absolute error is 37.94190979003906, q-error is 44.420223236083984
95% absolute error is 58.71411895751953, q-error is 95.15008544921875
********Epoch 10, training loss: 0.3353164993819937 || evaluation loss: ********


100%|████████████████████████████████████████| 200/200 [00:01<00:00, 125.62it/s]


50% absolute error is 3.5014660358428955, q-error is 4.121780872344971
90% absolute error is 34.95281982421875, q-error is 41.09223175048828
95% absolute error is 52.10723876953125, q-error is 86.98856353759766


KeyboardInterrupt: 

In [36]:
preds, labels = rnn.predict(eval_trace_df_sep, use_pre_info_only=False)

100%|████████████████████████████████████████| 200/200 [00:01<00:00, 148.49it/s]


50% absolute error is 10.830201148986816, q-error is 7.1018171310424805
90% absolute error is 91.40837097167969, q-error is 133.51956176757812
95% absolute error is 124.45758056640625, q-error is 311.17388916015625


In [44]:
preds2, labels2 = rnn2.predict(eval_trace_df_sep, use_pre_info_only=False)

100%|████████████████████████████████████████| 200/200 [00:01<00:00, 139.35it/s]


50% absolute error is 3.380521774291992, q-error is 4.0659284591674805
90% absolute error is 40.663795471191406, q-error is 47.04194641113281
95% absolute error is 59.644752502441406, q-error is 101.35169982910156


In [48]:
i = 210
idx = np.argsort(preds2[i])
print(len(idx))
np.stack((np.asarray(preds2[i])[idx], np.asarray(preds[i])[idx], np.asarray(labels2[i])[idx]), axis=1)

111


array([[  2.8291447,   7.1109896,  79.540054 ],
       [  2.8653407,   2.3412442,   3.416112 ],
       [  2.934177 ,   4.0760264,   4.0271263],
       [  2.9910045,   3.7841144,   3.8717294],
       [  3.0511217,   5.0034904,   3.1730235],
       [  3.148857 ,  18.968218 ,   3.1449356],
       [  3.170916 ,   6.0808353, 106.32712  ],
       [  3.183551 ,   6.5285897,   4.7487164],
       [  3.2096725,  33.221123 ,   2.8220966],
       [  3.2253332,   5.26631  ,   3.3115742],
       [  3.246746 ,   4.4229527,   5.8701944],
       [  3.2892797,   5.2571554,   7.374345 ],
       [  3.2927618,  24.601357 ,   2.9005537],
       [  3.3001895,   3.752159 ,   4.9782295],
       [  3.3236983,   3.4375095,   4.804597 ],
       [  3.3453178,   4.3794765,   3.0326128],
       [  3.4195132,   5.3936315,   8.1688   ],
       [  3.4844189,   6.0924673,   4.3422008],
       [  3.5340056,   4.0386086,   4.554105 ],
       [  3.5757537,   8.585522 ,  32.302185 ],
       [  3.5846162,   6.08425  ,   5.50

In [8]:
i = 210
idx = np.argsort(preds2[i])
print(len(idx))
np.stack((np.asarray(preds[i])[idx], np.asarray(preds2[i])[idx], np.asarray(labels2[i])[idx]), axis=1)

NameError: name 'preds2' is not defined

In [11]:
i = 2
idx = np.argsort(preds[i])
print(len(idx))
np.stack((np.asarray(preds[i])[idx], np.asarray(labels[i])[idx]), axis=1)

7


array([[  9.086283,  10.558472],
       [  9.326768,  10.257236],
       [ 16.062849,  39.89623 ],
       [ 20.419281,  69.41913 ],
       [ 35.720085,  52.07332 ],
       [ 44.310814, 110.97227 ],
       [ 50.8906  ,  45.67449 ]], dtype=float32)